## Notebook pour la récupération des commentaires des utilisateurs sur IMBD via Selenium

##  Download Library ⚙️

In [1]:
#!pip install -q lxml 
#pip install selenium

##  Import Library 📦

In [14]:
import time
import requests
import numpy as np
import pandas as pd
import urllib
from urllib import request
import bs4 
import lxml

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

!!! attention !!! Il faut télécharger un "chromedriver" de la même version que votre version chrome, et le mettre à coté de ce fichier jupyter dans les fichiers. Lien pour télécharger les chromedrivers: https://chromedriver.chromium.org/downloads

## I - Scraping sur une page donnée

Nous allons d'abord écrire une fonction qui va récupérer les informations et les commentaires à partir du lien de la page principale d'un film donné. Nous réutiliserons cette fonction dans une boucle pour "scrape" les films de notre liste de façon automatique.
Pour ce faire, nous avons commencé par travailler sur cette page: https://www.imdb.com/title/tt0499549/?ref_=ttls_li_tt

Globalement, notre code créer un robot qui va simuler l'utilisation du navigateur chrome. Dans notre fonction "scrape_imdb_reviews", nous dictons à notre robot de se rendre sur le lien que nous lui donnons en entrée. A partir de là, il va récuperer certaines informations sur le film, comme son titre, sa durée, sa note globale, son budget, ect.

Ensuite, nous lui demandons d'un peu scroller vers le bas pour faire apparaître un bouton "avis d'utilisateurs". Il clique dessus, clique sur le bouton "hide spoilers" pour retirer les commentaires avec une banière "warning: spoilers" (Ils sont plus difficiles à scraper et pas forcément important dans notre cas), puis il va aller cliquer plusieurs fois tout en bas des commentaires sur "load more" afin de charger plus de commentaires (25 de plus par "load more" cliqués). Le paramètre va "num_iterations" va en fait correspondre au nombre de fois où le robot va cliquer sur "load more". Le robot scrape ensuite tous les commentaires afffichés.

Enfin le robot nous retourne un dictionnaire avec pour clé le titre du film et pour valeurs une liste avec nos variables "scraped".

In [15]:
#fonctions que l'on va appeler ensuite pour récupèrer des éléments dans une page htlm avec selenium

def get_element_byclass(code_name, driver_wait):
    span_element = driver_wait.until(EC.presence_of_element_located((By.CLASS_NAME, code_name)))
    return span_element.text

def get_element_bycss(code_name, driver_wait):
    span_element = driver_wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, code_name)))
    return span_element.text

In [24]:
def scrape_imdb_reviews(url, with_driver = None, num_iterations=3, wait_time=20):
    
    if with_driver is None:
        driver = webdriver.Chrome()
    else:
        driver = with_driver
    driver.get(url)
    
    wait = WebDriverWait(driver, wait_time)
    
    #récupère le titre du film et son année de sortie
    title_htlm = wait.until(EC.presence_of_element_located((By.TAG_NAME, 'title')))
    title_imdb = title_htlm.get_attribute('text')
    title = title_imdb.replace(' - IMDb','') 
    print("Title:", title)
    
    #annee de sortie
    splitted = title.split()
    annee = splitted[-1].replace('(','').replace(')','')
    print(annee)
    
    #récupère la note globale du film
    note = get_element_byclass("sc-bde20123-1", wait)
    print(note)
    
    try:
        #récupère le budget estimé du film
        budget_ = get_element_bycss('.ipc-metadata-list__item[data-testid="title-boxoffice-budget"] .ipc-metadata-list-item__list-content-item', wait)
        budget = budget_.replace('\u202f','').replace(" $US(estimation)","")
        print(f"Budget: {budget}")
    
    #récupère le box-office
        recette_ = get_element_bycss('.ipc-metadata-list__item[data-testid="title-boxoffice-cumulativeworldwidegross"] .ipc-metadata-list-item__list-content-item', wait)
        recette = recette_.replace('\u202f','').replace(" $US", "")
        print(f"Cumulative Worldwide Gross: {recette}")
    except Exception:
        budget = np.nan
        recette = np.nan
    
    
    #récupère la durée du film
    duree_htlm = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'meta[property="og:description"]')))
    duree_ = duree_htlm.get_attribute('content')
    duree= duree_.split('|')[0].strip()
    print(duree)
    
    time.sleep(1)
    
    #Il faut scroller vers le bas pour pouvoir cliquer sur le bouton "avis d'utilisateurs"
    
    # Get the current scroll position
    current_scroll_position = driver.execute_script("return window.scrollY;")
    # Scroll down by 400 pixels from the current position
    scroll_by_pixels = 400
    new_scroll_position = current_scroll_position + scroll_by_pixels
    driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")
    time.sleep(1.5)
    
    
    try:
        #cherche le bouton "avis d'utilsateurs" et clique de dessus.
        user_comments_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.three-Elements > span.label")))
        user_comments_button.click()
      
        #cherche le bouton "Hide Spoilers" et clique sur ce bouton
        spoiler_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "faceter-facets-text")))
        spoiler_button.click()
    
        try:
            # Clique sur le bouton "Load More" plusieurs fois
            for i in range(num_iterations):
                load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
                load_more_button.click()
                driver.implicitly_wait(3)

            time.sleep(2)
        
        finally:
            # Trouve les commentaires par classe CSS
            reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")
            reviews_txt = [review.text for review in reviews]
 
        # Imprime le nombre total de commentaires récupérés
        print(f"Nombre total de commentaires : {len(reviews)}")
    
    except Exception:
        reviews_txt = np.nan

    time.sleep(2)

    # Ferme le navigateur si pas de driver en entré
    if with_driver is None:
        driver.quit()

    return {title: [reviews_txt, annee, note, budget, recette, duree]}

Test sur le film avatar: https://www.imdb.com/title/tt0499549/

In [6]:
url_avatar = "https://www.imdb.com/title/tt0499549/"
df_avatar = scrape_imdb_reviews(url_avatar, num_iterations=3, wait_time=20)

Title: Avatar (2009)
2009
7,9
Budget: 237000000
Cumulative Worldwide Gross: 2923706026
2h 42m
Nombre total de commentaires : 100


In [7]:
# vérification
#print les 2 premiers commentaires
df_avatar['Avatar (2009)'][0][:2]

["I was waiting for this day for the longest time. I was a kid back in 2009 when this movie released. So never got to watch it back then. But now when it rereleased I booked the first day show to a IMAX show and oh boy was I blown away! This is nothing short of a masterpiece! It's beyond belief how a film like this could've been made. Every scene, every shot is perfection. You are transferred to a different world and become so engrossed in the film. Never have I attended a movie where at the end of it people got up from their seats and started clapping! Last time this happened was after Infinity War. That movie too was a damn brilliant one. I'm from India and this is legit rare, where people go crazy, so crazy after any English film. This was one experience that I will never forget. I'm definitely going for it again next weekend cuz just once was not enough.\n\nEvery human on the planet needs to experience this magnificent work of art!",
 "Forgive me, I'm going to jump from professiona

In [8]:
#vérification que nous avons pas de commentaire "vide"
X = np.array([len(review) for review in df_avatar["Avatar (2009)"][0]])
print(len(X))
print(X)

100
[ 945 5506 2389 4167 1033 2754 5675 1922 2022 1352 3585 3852 2429  850
  997 1305  715 1047  184 1724  824 1275 2433 4197 1014 5140 1856 2332
 2867 2324 5172 1030 4866 2689  184 2714 2715 4956 1662 2300  820 4488
  767 2884 2559 4741 2492 1159 1759 1268 5574 1679 3027  992  765 4081
  426 1418  820 1440 5216  397  753  296  550  802 1259  743 1211 2292
  792 1416 1578 2094 3903  403  972 1263 1049 5415 3298  903  484 2565
  781 1482  477 1119  719 2289 3680  817 2203 1361  254  769 1792 1039
 3628  227]


In [71]:
#Test pour un autre film, sur une page htlm que nous avions pas vu
url_cars = "https://www.imdb.com/title/tt0317219/?ref_=nv_sr_srsg_0_tt_8_nm_0_q_cars"
reviews_cars = scrape_imdb_reviews(url_cars, num_iterations=3, wait_time=20)

Title: Cars - Quatre roues (2006)
2006
7,2
Budget: 120 000 000 $US(estimation)
Cumulative Worldwide Gross: 461 991 867 $US
1h 57m
Nombre total de commentaires : 100


En faisant tourner la grosse fonction "scrap_review_all" (en dessous), notre fonction s'est arrêtés sur le film "La bataille du lac Changjin" (https://www.imdb.com/title/tt13462900/?ref_=ttls_li_tt). En effet, sur la page du film, il n'y a pas le bouton "metascore" près du bouton "avis d'utilisateurs", ce qui fausse notre algorithme. Nous avons donc modifié notre fonction "scrape_imdb_reviews" en conséquence en mettant un systeme de "try - except " afin que notre fonction continue malgré tout. Les commentaires ne peuvent donc pas être scrapés sur ce genre de page avec notre algorithme (Cela pourrait être modifié mais ce problème concerne très peu de film) et nous remplaçons les commentaires initialement attendus par un "np.nan".

In [9]:
#vérification
url_changjin = "https://www.imdb.com/title/tt13462900/?ref_=ttls_li_tt"
reviews_changjin = scrape_imdb_reviews(url_changjin, num_iterations=7, wait_time=20)
print(reviews_changjin.items())

Title: La bataille du lac Changjin (2021)
2021
5,5
Budget: 200000000
Cumulative Worldwide Gross: 902548476
2h 56m
dict_items([('La bataille du lac Changjin (2021)', [nan, '2021', '5,5', '200000000', '902548476', '2h 56m'])])


In [25]:
url_chinois = "https://www.imdb.com/title/tt13364790/?ref_=ttls_li_tt"
reviews_chinois = scrape_imdb_reviews(url_chinois, num_iterations=7, wait_time=20)
print(reviews_chinois.items())

Title: Ni hao, Li Huanying (2021)
2021
6,9
2h 8m
dict_items([('Ni hao, Li Huanying (2021)', [nan, '2021', '6,9', nan, nan, '2h 8m'])])


## II - Scraping sur plusieurs page avec une boucle

Afin de pouvoir accéder aux films du top box office avec une boucle, il faut récupérer les liens pour chaque film. Pour ce faire, nous allons nous servir de ce site qui liste le top box office: https://www.imdb.com/list/ls098063263/. En le "scrapant" avec Beautifulsoup, nous pouvons récupérer les liens, ou plutôt les id dans les liens pour chaque film et ainsi faire une boucle. Le lien se retrouve avec "https://www.imdb.com" puis l'id du film puis "?ref_=tts_li_i". 

In [17]:
#scraping de la page avec beautifulsoup.
main_url = 'https://www.imdb.com/list/ls098063263/'
request_text = request.urlopen(main_url).read()
page = bs4.BeautifulSoup(request_text, "lxml")

In [29]:
def scrap_review_all(wait_time=10, num_iterations=3):
    # Initialise le navigateur
    driver = webdriver.Chrome()
    
    #créer un dico vide
    dico = {}
    
    #va récupérer les id dans les liens des films
    h3_tags = page.find_all('h3', class_='lister-item-header')
    i=1
    for h3_tag in h3_tags:
        a_tag = h3_tag.find('a')
        
        #id du film
        href_value = a_tag['href']
        
        #récupère l'url du film qui nous interesse avec l'id
        url = 'https://www.imdb.com'+href_value+'?ref_=ttls_li_i'
        
        dico.update(scrape_imdb_reviews(url, with_driver=driver, num_iterations=num_iterations, wait_time= wait_time))
        
        print(i)
        i+=1
        
        time.sleep(0.5)
    #ferme le navigateur
    driver.quit()
    return dico
        

In [31]:
dico_all = scrap_review_all(wait_time=20, num_iterations=7)

Title: Avatar (2009)
2009
7,9
Budget: 237000000
Cumulative Worldwide Gross: 2923706026
2h 42m
Nombre total de commentaires : 200
1
Title: Avengers: Endgame (2019)
2019
8,4
Budget: 356000000
Cumulative Worldwide Gross: 2799439100
3h 1m
Nombre total de commentaires : 200
2
Title: Avatar : La Voie de l'eau (2022)
2022
7,6
Budget: 350000000
Cumulative Worldwide Gross: 2320250281
3h 12m
Nombre total de commentaires : 199
3
Title: Titanic (1997)
1997
7,9
Budget: 200000000
Cumulative Worldwide Gross: 2264743305
3h 14m
Nombre total de commentaires : 198
4
Title: Star Wars : Épisode VII - Le Réveil de la Force (2015)
2015
7,8
Budget: 245000000
Cumulative Worldwide Gross: 2071310218
2h 18m
Nombre total de commentaires : 199
5
Title: Avengers: Infinity War (2018)
2018
8,4
Budget: 321000000
Cumulative Worldwide Gross: 2052415039
2h 29m
Nombre total de commentaires : 200
6
Title: Spider-Man: No Way Home (2021)
2021
8,2
Budget: 200000000
Cumulative Worldwide Gross: 1921847111
2h 28m
Nombre total de 

Nombre total de commentaires : 200
56
Title: Le Roi Lion (1994)
1994
8,5
Budget: 45000000
Cumulative Worldwide Gross: 968511805
1h 28m
Nombre total de commentaires : 199
57
Title: Le Livre de la jungle (2016)
2016
7,4
Budget: 175000000
Cumulative Worldwide Gross: 967724775
1h 46m
Nombre total de commentaires : 200
58
Title: Le Hobbit: La Bataille des Cinq Armées (2014)
2014
7,4
Budget: 250000000
Cumulative Worldwide Gross: 962201338
2h 24m
Nombre total de commentaires : 200
59
Title: Pirates des Caraïbes : Jusqu'au bout du monde (2007)
2007
7,1
Budget: 300000000
Cumulative Worldwide Gross: 961691209
2h 49m
Nombre total de commentaires : 200
60
Title: Le Hobbit: La Désolation de Smaug (2013)
2013
7,8
Budget: 225000000
Cumulative Worldwide Gross: 959027992
2h 41m
Nombre total de commentaires : 200
61
Title: Doctor Strange in the Multiverse of Madness (2022)
2022
6,9
Budget: 200000000
Cumulative Worldwide Gross: 955775804
2h 6m
Nombre total de commentaires : 200
62
Title: Oppenheimer (202

!!! La cellule ci-dessus prend environ 40 min à s'éxécuter. Assurer vous d'avoir du temps et une bonne connection pour éviter tous problèmes.

def scrap_review_all(movie_titles, release_year, wait_time=10, num_iterations=3, browser="chrome"):
    if browser == "chrome":
        driver = webdriver.Chrome()
    else:
        # Add support for other browsers if needed
        raise ValueError("Unsupported browser")
    
    dico_all_comments = {}
    driver.get("https://www.google.com")
    wait = WebDriverWait(driver, wait_time)
    # Attendre que le bouton soit présent
    button = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='QS5gu sy4vM' and text()='Tout refuser']")))
        
    # Cliquez sur le bouton
    button.click()
    
    for i in range(len(movie_titles)):
        # Ouvrir Google
        
        cleaned_title = movie_titles[i].strip().replace("\xa0", " ")
        driver.get("https://www.google.com")
        wait = WebDriverWait(driver, wait_time)
        
        
        
        search_box = driver.find_element(By.NAME, "q")
        search_box.send_keys("imdb title " + cleaned_title + " " + str(release_year[i]))
        print("imdb title " + cleaned_title + " " + str(release_year[i]))
        search_box.send_keys(Keys.RETURN)
        
        # Attente que les résultats de recherche soient chargés
        wait.until(EC.presence_of_element_located((By.ID, "search")))
        
        time.sleep(1.5)
        
        # Trouver le premier lien dans les résultats de recherche et cliquer dessus
        first_link = driver.find_element(By.CSS_SELECTOR, "h3")
        first_link.click()
        
        time.sleep(1)
        
        # Locate the span element with the specified class
        span_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sc-bde20123-1")))

        # Get the text content of the span element
        note = span_element.text

        #  Print or use the extracted text
        print(note)
    
        time.sleep(1.5)

        # Get the current scroll position
        current_scroll_position = driver.execute_script("return window.scrollY;")

           # Scroll down by 200 pixels from the current position
        scroll_by_pixels = 400
        new_scroll_position = current_scroll_position + scroll_by_pixels
        driver.execute_script(f"window.scrollTo(0, {new_scroll_position});")
    
        time.sleep(2)
    
        user_comments_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.three-Elements > span.label")))

         # Cliquez sur le bouton
        user_comments_button.click()
        wait = WebDriverWait(driver, wait_time)
        
        time.sleep(1)
    
    
         #cherche le bouton "Hide Spoilers"
        spoiler_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "faceter-facets-text")))
        # Cliquez sur le bouton
        spoiler_button.click()
        
 
        # Clique sur le bouton "Load More" plusieurs fois
        for i in range(num_iterations):
            load_more_button = wait.until(EC.element_to_be_clickable((By.ID, "load-more-trigger")))
            load_more_button.click()
            time.sleep(2)

        time.sleep(3)

        # Trouve les commentaires par classe CSS
        reviews = driver.find_elements(By.CSS_SELECTOR, ".text.show-more__control")

        print( cleaned_title)
        reviews_txt = [review.text for review in reviews]
        dico_all_comments[cleaned_title] = [reviews_txt, note]
        time.sleep(3)
    
    driver.quit()
    return dico_all_comments

Nous vérifions que nous avons bien récupérer ce qu'on voulait:

In [32]:
print(dico_all.keys())
print(len(dico_all.keys()))
for i in dico_all.keys():
    print( len(dico_all[i][0]))

dict_keys(['Avatar (2009)', 'Avengers: Endgame (2019)', "Avatar : La Voie de l'eau (2022)", 'Titanic (1997)', 'Star Wars : Épisode VII - Le Réveil de la Force (2015)', 'Avengers: Infinity War (2018)', 'Spider-Man: No Way Home (2021)', 'Jurassic World (2015)', 'Le Roi Lion (2019)', 'Avengers (2012)', 'Fast & Furious 7 (2015)', 'Top Gun: Maverick (2022)', 'La Reine des Neiges 2 (2019)', 'Barbie (2023)', "Avengers : L'Ère d'Ultron (2015)", 'Super Mario Bros. le film (2023)', 'Black Panther (2018)', 'Harry Potter et les Reliques de la Mort : partie 2 (2011)', 'Star Wars : Épisode VIII - Les Derniers Jedi (2017)', 'La reine des neiges (2013)', 'Jurassic World: Fallen Kingdom (2018)', 'La Belle et la Bête (2017)', 'Les Indestructibles 2 (2018)', 'Fast & Furious 8 (2017)', 'Iron Man 3 (2013)', 'Les Minions (2015)', 'Le Seigneur des anneaux : Le Retour du roi (2003)', 'Captain America: Civil War (2016)', 'Aquaman (2018)', 'Skyfall (2012)', 'Spider-Man: Far from Home (2019)', 'Captain Marvel (2

TypeError: object of type 'float' has no len()

Il y a bien nos 100 films, avec chacun environ 200 commentaires. Certains ont 175 commentaires "seulement" à cause de problèmes de connection un peu aléatoire pour charger les pages. Il est peut-être possible d'arranger ce problème en laissant plus de temps à l'algorithme du dessus. Mais dans notre cas, ce problème concerne environ 5 films sur les 100. Donc nous allons nous contenter de cela.

In [33]:
data_all = pd.DataFrame(dico_all)

In [34]:
data_all.to_json('data.json')